In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import pairwise_distances
from scipy.spatial.distance import cosine

In [22]:
class RelatedLessons:
    def __init__(self, tf_idf_matrix_file, n_lessons=5):
        self.parpararagraph_tf_idf = pd.read_csv(tf_idf_matrix_file).iloc[:, 1:]
        self.pararagraph_ids = self.parpararagraph_tf_idf['id'].values
        self.parpararagraph_tf_idf.drop('id', axis=1, inplace=True)
        self.n_lessons = n_lessons
        self.distance_matrix = self.calculate_distance_matrix(self.parpararagraph_tf_idf)
        
    def calculate_distance_matrix(self, tf_idf_matrix):
        distances = pairwise_distances(self.parpararagraph_tf_idf.iloc[:,1:], metric = 'cosine', n_jobs = -1)
        return pd.DataFrame(distances)
    
    def get_n_related_lessons(self, index):
        return self.distance_matrix.loc[index].sort_values().head(self.n_lessons+1).index[1:].tolist()
    
    def run(self):
        related_lessons = []
        for index, par_id in enumerate(self.pararagraph_ids):
            related_indexes = [int(i) for i in self.get_n_related_lessons(index)]
            source_lesson_id = par_id
            related_lesson_ids = [self.pararagraph_ids[related_index] for related_index in related_indexes]
            related_lessons.append([source_lesson_id,related_lesson_ids])
        return pd.DataFrame(related_lessons, columns=['source_lesson', 'related_lessons'])

In [23]:
related_lessons_obj = RelatedLessons("./data/data_version_3/related_lessons/tfidf.csv")

In [24]:
related_lessons_df = related_lessons_obj.run()

# Check if related lessons make sens

In [37]:
def get_lessons(par_ids, id_par_map):
    for par_id in par_ids:
        print(id_par_map[id_par_map['id'] == par_id]["paragraph"].values)

id_par_map = pd.read_csv('./data/data_version_3/related_lessons/lessons_ids_paragraphs.csv').iloc[:, 1:]
for i in range(5):
    par_id = related_lessons_df.iloc[i]['source_lesson']
    related_ids = related_lessons_df.iloc[i]['related_lessons']
    
    print("Source Lesson")
    print(par_id)
    get_lessons([par_id], id_par_map)

    print("===Related Lessons")
    print(related_ids)
    get_lessons(related_ids, id_par_map)
    print()

Source Lesson
1
['Realistic sector and institutional assessments. Implementation of the soft components of infrastructure projects—such as gender equality, safeguards, or capacity building—financed by development organizations in Azerbaijan is often challenging because of the lack of capacity and motivation of the executing agencies. It is important to ensure in-depth assessment of the country and sector context as part of transaction technical assistance to guide the design of the soft components of public utility projects in a well contextualized manner. Weak planning systems; business processes; and technical, administration, and financial capacity of the executing agencies can also put project outcomes at risk. Efficiently tailored technical assistance and closer supervision could have helped mitigate this risk.']
===Related Lessons
[7, 3187, 2882, 5, 21956]
['Technical assistance for institutional and regulatory issues. Technical assistance should be attached to the project to sup